In [1]:
# Import necessary libraries;
import numpy as np
from numpy import mean
from numpy import std


import pandas as pd

import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set_style('white')

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


from mpl_toolkits.mplot3d import Axes3D
from numpy import genfromtxt

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.datasets import load_digits, make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler 
from sklearn.preprocessing import RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn import tree
from random import choices


In [2]:
fireWall = pd.read_csv('log2.csv')

fireWallXData = fireWall[['Source Port', 'Destination Port','NAT Source Port',
                         'NAT Destination Port','Bytes','Bytes Sent','Bytes Received','Packets',
                        'Elapsed Time (sec)','pkts_sent','pkts_received']]
fireWallyData = fireWall[['Action']]

X = fireWallXData.iloc[:,0:10]
y = fireWallyData.iloc[:,0]


print(y.shape)
print(X.shape)

#print(fireWall.shape)

FireWALLData = pd.concat([X, y], axis=1)
print(FireWALLData.shape)

newX = FireWALLData.iloc[:,0:9]
newy = FireWALLData.iloc[:,10]

from sklearn.preprocessing import LabelEncoder
encodeLabel = LabelEncoder()
newy = encodeLabel.fit_transform(newy)
print(newy)


FireWALLData.loc[FireWALLData['Action'] == 0, 'Action'] = 1
FireWALLData.loc[FireWALLData['Action'] == 1, 'Action'] = 0
FireWALLData.loc[FireWALLData['Action'] == 2, 'Action'] = 1
FireWALLData.loc[FireWALLData['Action'] == 3, 'Action'] = 0

newy = FireWALLData.iloc[:,10]

FireWALLData.head()




(65532,)
(65532, 10)
(65532, 11)
[0 0 0 ... 2 2 2]


,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,Action
0,57222,53,54587,53,177,94,83,2,30,1,allow
1,56258,3389,56258,3389,4768,1600,3168,19,17,10,allow
2,6881,50321,43265,50321,238,118,120,2,1199,1,allow
3,50553,3389,50553,3389,3327,1438,1889,15,17,8,allow
4,50002,443,45848,443,25358,6778,18580,31,16,13,allow


In [3]:


X_train, X_test, y_train, y_test = train_test_split( newX, newy, test_size=60000, random_state=1738)


# SUPPORT VECTOR MACHINE TRIAL ONE ON Fire Wall  DATASET

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)    
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_1_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL ONE RESULTS ---------------------------------------------")

Fitting 2 folds for each of 324 candidates, totalling 648 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL ONE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 648 out of 648 | elapsed:   32.5s finished


In [10]:
TRIAL_1_SVM.cv_results_

{'mean_fit_time': array([0.92504489, 0.91528618, 0.92029095, 0.90527797, 0.91153359,
        0.93830705, 0.94105828, 0.99035168, 1.00436354, 0.93805623,
        0.9443121 , 0.93305242, 0.9140352 , 0.92629647, 0.95982563,
        0.95006669, 0.87925601, 0.88175786, 0.92729664, 0.92654669,
        0.93305218, 0.94881582, 0.9480654 , 0.94030881, 0.77791893,
        0.46865249, 0.4791621 , 0.95407057, 0.97083497, 0.93130088,
        0.98159397, 0.94556308, 0.77616751, 0.4521389 , 0.25747144,
        0.37582326, 0.92604637, 0.95031679, 0.98159349, 0.9593246 ,
        0.7676599 , 0.46715152, 0.26347649, 0.15163028, 0.36456358,
        0.96557987, 0.96532989, 0.96708167, 0.77741826, 0.47390711,
        0.29225171, 0.17840326, 0.24070668, 0.4373759 , 0.9788419 ,
        0.96457934, 0.77841926, 0.46990407, 0.3170228 , 0.26172507,
        0.24796355, 0.3773241 , 0.50843728, 0.97408748, 0.80018806,
        0.47916198, 0.31977499, 0.29750597, 0.30601346, 0.29300201,
        0.4040972 , 0.49492645,

In [11]:
print("---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_1_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 100,
 'classifier__gamma': 1.0,
 'classifier__kernel': 'rbf'}

In [12]:
print("---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------")
TRIAL_1_SVM.cv_results_['rank_test_f1_micro']
TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 100000,
 'classifier__gamma': 1.0,
 'classifier__kernel': 'rbf'}

In [13]:
print("---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------")
TRIAL_1_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10,
 'classifier__gamma': 0.01,
 'classifier__kernel': 'rbf'}

In [14]:
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 1 svm using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 1 svm using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 1 svm using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 1 svm using best NEG LOG LOSS hyperparameters :0.9919166666666667
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 svm using best F1 hyperparameters :0.9955333333333334
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 svm using best ROC_AUC hyperparameters :0.9725333333333334
------------------------------------------------------------------------------------------------


# SUPPORT VECTOR MACHINE TRIAL TWO ON Fire Wall  DATASET

In [15]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', MinMaxScaler(feature_range = (0, 1))),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_2_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL TWO RESULTS ---------------------------------------------")

Fitting 2 folds for each of 324 candidates, totalling 648 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL TWO RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 648 out of 648 | elapsed:   30.5s finished


In [16]:
TRIAL_2_SVM.cv_results_

{'mean_fit_time': array([0.70335436, 0.91778934, 0.91553688, 0.90627933, 0.89827192,
        0.8987726 , 0.90127444, 0.9187901 , 0.93830681, 0.73863423,
        0.9192903 , 0.91103208, 0.91278422, 0.90903139, 0.89101601,
        0.93155086, 1.00136125, 0.95957601, 0.81870306, 0.97433758,
        0.94030905, 0.94205928, 0.98359597, 1.0418967 , 1.053406  ,
        0.86899674, 0.4989295 , 0.75439847, 0.97058463, 0.97909307,
        0.95582175, 1.01987696, 0.98734915, 0.8392216 , 0.49642682,
        0.24746311, 0.7351321 , 0.95256877, 0.93705559, 0.96958351,
        0.95732284, 0.81745291, 0.4951756 , 0.26497781, 0.15863633,
        0.70160341, 0.94481266, 0.94831574, 0.97458827, 0.81595159,
        0.48942101, 0.29425323, 0.18065524, 0.23545265, 0.70961022,
        0.95407069, 0.94606328, 0.82871282, 0.48466671, 0.29375279,
        0.22869694, 0.26773036, 0.30551243, 0.70860934, 0.98684859,
        0.82245743, 0.50268209, 0.31101763, 0.25672066, 0.34504676,
        0.35805857, 0.4781611 ,

In [17]:
print("---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_2_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 100000,
 'classifier__gamma': 0.1,
 'classifier__kernel': 'rbf'}

In [18]:
print("---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------")
TRIAL_2_SVM.cv_results_['rank_test_f1_micro']
TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10000,
 'classifier__gamma': 1.0,
 'classifier__kernel': 'rbf'}

In [19]:
print("---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------")
TRIAL_2_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10,
 'classifier__gamma': 1.0,
 'classifier__kernel': 'poly'}

In [20]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 2 svm using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 2 svm using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 2 svm using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 2 svm using best NEG LOG LOSS hyperparameters :0.9894
------------------------------------------------------------------------------------------------
Accuracy of Trial 2 svm using best F1 hyperparameters :0.9917
------------------------------------------------------------------------------------------------
Accuracy of Trial 2 svm using best ROC_AUC hyperparameters :0.9652166666666666
------------------------------------------------------------------------------------------------


# SUPPORT VECTOR MACHINE TRIAL THREE ON Fire Wall  DATASET

In [21]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    # Run GridSearch to find best hyperparameters per 5000 data points 
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_3_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL THREE RESULTS ---------------------------------------------")

Fitting 2 folds for each of 324 candidates, totalling 648 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL THREE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 648 out of 648 | elapsed:   28.7s finished


In [22]:
TRIAL_3_SVM.cv_results_

{'mean_fit_time': array([0.92854834, 0.88501084, 0.91028178, 0.86999738, 0.92029011,
        0.91428614, 0.90652871, 0.94856536, 0.96633077, 0.90677953,
        0.90928137, 0.92604649, 0.92429507, 0.89526975, 0.93330264,
        0.92429507, 0.91303456, 0.91628778, 0.93080044, 0.93630493,
        0.88851404, 0.95181859, 1.00836682, 0.96983445, 0.76215518,
        0.45514107, 0.46540022, 0.91728854, 0.89677107, 0.91178417,
        0.92079151, 0.93130088, 0.7363832 , 0.44012797, 0.25021541,
        0.37031841, 0.90202558, 0.90352714, 0.93205166, 0.92029154,
        0.7493943 , 0.45539153, 0.24496055, 0.16789436, 0.34854937,
        0.94906616, 0.92929935, 0.94356143, 0.73738408, 0.45313954,
        0.29500365, 0.18040478, 0.18791151, 0.37206984, 0.9458133 ,
        0.91678798, 0.74514091, 0.4506377 , 0.2837441 , 0.22769535,
        0.21168292, 0.29600453, 0.43011987, 0.95181811, 0.75715065,
        0.46139681, 0.30125904, 0.23370004, 0.24370933, 0.27048254,
        0.3460474 , 0.48841989,

In [23]:
print("---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_3_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 100,
 'classifier__gamma': 1.0,
 'classifier__kernel': 'rbf'}

In [24]:
print("---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------")
TRIAL_3_SVM.cv_results_['rank_test_f1_micro']
TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 100000,
 'classifier__gamma': 1.0,
 'classifier__kernel': 'rbf'}

In [25]:
print("---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------")
TRIAL_3_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10000,
 'classifier__gamma': 0.1,
 'classifier__kernel': 'linear'}

In [26]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 3 svm using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 3 svm using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 3 svm using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 3 svm using best NEG LOG LOSS hyperparameters :0.9919166666666667
------------------------------------------------------------------------------------------------
Accuracy of Trial 3 svm using best F1 hyperparameters :0.9955333333333334
------------------------------------------------------------------------------------------------
Accuracy of Trial 3 svm using best ROC_AUC hyperparameters :0.8505166666666667
------------------------------------------------------------------------------------------------


# SUPPORT VECTOR MACHINE TRIAL FOUR ON Fire Wall  DATASET

In [27]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    # Run GridSearch to find best hyperparameters per 5000 data points 
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_4_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FOUR RESULTS ---------------------------------------------")

Fitting 2 folds for each of 324 candidates, totalling 648 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL FOUR RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 648 out of 648 | elapsed:   28.8s finished


In [28]:
TRIAL_4_SVM.cv_results_

{'mean_fit_time': array([0.89076567, 0.88350928, 0.88626063, 0.87575197, 0.88651156,
        0.90227461, 0.92354333, 0.92929912, 0.99085164, 0.91678786,
        0.90452743, 0.88951468, 0.89326823, 0.91178381, 0.92654669,
        0.92479539, 0.85298347, 0.8514812 , 0.90202546, 0.89777195,
        0.90402734, 0.91428614, 0.93630493, 0.92729771, 0.75489938,
        0.42686725, 0.45063758, 0.91228449, 0.93605506, 0.92754745,
        0.97984302, 0.94781506, 0.77566707, 0.44463193, 0.24295866,
        0.38257909, 0.91954052, 0.92429495, 0.97133541, 0.9433111 ,
        0.74689245, 0.46439898, 0.2429589 , 0.15838599, 0.3775748 ,
        0.99935949, 1.00836718, 0.96457958, 0.77641785, 0.47090483,
        0.28724682, 0.17665184, 0.24871361, 0.42261338, 0.97809136,
        0.94381142, 0.76440728, 0.44663417, 0.27573717, 0.21693647,
        0.21768689, 0.31326962, 0.51068926, 0.95932484, 0.76240528,
        0.45764327, 0.28224266, 0.26147497, 0.24721241, 0.25847268,
        0.35105157, 0.52695334,

In [29]:
print("---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_4_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 100,
 'classifier__gamma': 1.0,
 'classifier__kernel': 'rbf'}

In [30]:
print("---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------")
TRIAL_4_SVM.cv_results_['rank_test_f1_micro']
TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10000,
 'classifier__gamma': 1.0,
 'classifier__kernel': 'rbf'}

In [31]:
print("---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------")
TRIAL_4_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 10000,
 'classifier__gamma': 1e-08,
 'classifier__kernel': 'sigmoid'}

In [32]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 4 svm using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 4 svm using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 4 svm using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 4 svm using best NEG LOG LOSS hyperparameters :0.9919166666666667
------------------------------------------------------------------------------------------------
Accuracy of Trial 4 svm using best F1 hyperparameters :0.9938666666666667
------------------------------------------------------------------------------------------------
Accuracy of Trial 4 svm using best ROC_AUC hyperparameters :0.5776166666666667
------------------------------------------------------------------------------------------------


# SUPPORT VECTOR MACHINE TRIAL FIVE ON Fire Wall  DATASET

In [33]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    # Run GridSearch to find best hyperparameters per 5000 data points
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_5_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FIVE RESULTS ---------------------------------------------")

Fitting 2 folds for each of 324 candidates, totalling 648 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL FIVE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 648 out of 648 | elapsed:   29.8s finished


In [34]:
TRIAL_5_SVM.cv_results_

{'mean_fit_time': array([0.89326799, 0.90853107, 0.87450218, 0.87500215, 0.8732506 ,
        0.90152419, 0.90002346, 0.94080853, 0.97408712, 0.92054105,
        0.89501905, 0.89301777, 0.89126611, 0.92429495, 0.92354405,
        0.92604637, 0.8574872 , 0.87425172, 0.89401817, 0.89827204,
        0.89176667, 0.92129219, 0.933303  , 0.91678834, 0.75715101,
        0.46139669, 0.49067187, 0.91879046, 0.8897655 , 0.93405306,
        0.92579615, 0.92604673, 0.73313141, 0.45213854, 0.26072443,
        0.38833368, 0.89977384, 0.91353512, 0.94130933, 0.94656372,
        0.74163747, 0.46514976, 0.26447749, 0.16864502, 0.37907588,
        0.93980789, 0.94331133, 0.93980861, 0.73938608, 0.4666512 ,
        0.3032608 , 0.17565084, 0.26873136, 0.44087923, 0.95131803,
        0.93180132, 0.75314772, 0.46264791, 0.31276906, 0.24571133,
        0.25396848, 0.33453774, 0.41911054, 0.94055867, 0.76240587,
        0.46214712, 0.32202709, 0.29925752, 0.30851519, 0.34679842,
        0.39959335, 0.49192262,

In [35]:
print("---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_5_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 100,
 'classifier__gamma': 1.0,
 'classifier__kernel': 'rbf'}

In [36]:
print("---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------")
TRIAL_5_SVM.cv_results_['rank_test_f1_micro']
TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 100000,
 'classifier__gamma': 1.0,
 'classifier__kernel': 'rbf'}

In [37]:
print("---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------")
TRIAL_5_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------


{'classifier': SVC(max_iter=7000, probability=True),
 'classifier__C': 100000,
 'classifier__gamma': 1.0,
 'classifier__kernel': 'rbf'}

In [38]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 5 svm using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 5 svm using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 5 svm using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 5 svm using best NEG LOG LOSS hyperparameters :0.9919166666666667
------------------------------------------------------------------------------------------------
Accuracy of Trial 5 svm using best F1 hyperparameters :0.9955333333333334
------------------------------------------------------------------------------------------------
Accuracy of Trial 5 svm using best ROC_AUC hyperparameters :0.9955333333333334
------------------------------------------------------------------------------------------------
